### Library

In [942]:
import torch
import os
import torch.optim as optim
import numpy as np
import seaborn as sns
import torchvision
import pandas as pd
from torchvision import transforms, datasets
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary
import matplotlib as plt
from torch.utils.data import DataLoader, SubsetRandomSampler

In [943]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

device(type='cuda', index=0)

In [944]:
data_dir = './Data'

### Metrics

In [945]:
e_pochs = 100
l_r=0.0001
batch_size = 8
criter = nn.CrossEntropyLoss()


### Data processing


In [946]:
dataset = datasets.ImageFolder(data_dir)
dataset.class_to_idx

{'agronyem_repens': 0,
 'ailanto': 1,
 'arrenatereto': 2,
 'artemisia_verlotorum': 3,
 'populus_nigra': 4,
 'rubus_caesius': 5,
 'ulmus_minor': 6}

In [947]:
size = len(dataset)
indices = list(range(size))
np.random.shuffle(indices)

In [948]:
num_class =len(dataset.classes)
num_class

7

In [949]:
class_image_counts = {c: 0 for c in dataset.classes}
for img, label in dataset:
    class_image_counts[dataset.classes[label]] += 1
for c, count in class_image_counts.items():
    print(f"Class '{c}' contains {count} images.")


Class 'agronyem_repens' contains 119 images.
Class 'ailanto' contains 121 images.
Class 'arrenatereto' contains 82 images.
Class 'artemisia_verlotorum' contains 139 images.
Class 'populus_nigra' contains 128 images.
Class 'rubus_caesius' contains 102 images.
Class 'ulmus_minor' contains 91 images.


In [950]:
dir(transforms)

['AugMix',
 'AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'ElasticTransform',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandAugment',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 'TrivialAugmentWide',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional_pil',
 '_functional_tensor',
 '_presets',
 'autoaugment',
 'functional',
 'transforms']

In [951]:
preprocess = transforms.Compose([
    transforms.Resize((256,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

In [952]:
data = datasets.ImageFolder(root = data_dir, transform = preprocess)


In [953]:
test_split_index = int(np.floor(0.2 * size))
test_split_index

156

In [954]:
test_idx  = indices[:test_split_index]
train_idx = indices[test_split_index:]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler  = SubsetRandomSampler(test_idx)

In [955]:
train_loader = DataLoader(dataset=data, shuffle=False, batch_size=8, num_workers=4)
     

In [956]:
data.class_to_idx.items()

dict_items([('agronyem_repens', 0), ('ailanto', 1), ('arrenatereto', 2), ('artemisia_verlotorum', 3), ('populus_nigra', 4), ('rubus_caesius', 5), ('ulmus_minor', 6)])

In [957]:
idx_to_class = {v: k for k, v in data.class_to_idx.items()}
def get_class_distribution_loaders(dataloader):
  count_dict = {k:0 for k,v in data.class_to_idx.items()}
  for input, label in dataloader:
      label = label.item()
      label = idx_to_class[label]
      count_dict[str(label)] += 1    
  return count_dict

### Models


In [958]:
dir(models)

['AlexNet',
 'AlexNet_Weights',
 'ConvNeXt',
 'ConvNeXt_Base_Weights',
 'ConvNeXt_Large_Weights',
 'ConvNeXt_Small_Weights',
 'ConvNeXt_Tiny_Weights',
 'DenseNet',
 'DenseNet121_Weights',
 'DenseNet161_Weights',
 'DenseNet169_Weights',
 'DenseNet201_Weights',
 'EfficientNet',
 'EfficientNet_B0_Weights',
 'EfficientNet_B1_Weights',
 'EfficientNet_B2_Weights',
 'EfficientNet_B3_Weights',
 'EfficientNet_B4_Weights',
 'EfficientNet_B5_Weights',
 'EfficientNet_B6_Weights',
 'EfficientNet_B7_Weights',
 'EfficientNet_V2_L_Weights',
 'EfficientNet_V2_M_Weights',
 'EfficientNet_V2_S_Weights',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'GoogLeNet_Weights',
 'Inception3',
 'InceptionOutputs',
 'Inception_V3_Weights',
 'MNASNet',
 'MNASNet0_5_Weights',
 'MNASNet0_75_Weights',
 'MNASNet1_0_Weights',
 'MNASNet1_3_Weights',
 'MaxVit',
 'MaxVit_T_Weights',
 'MobileNetV2',
 'MobileNetV3',
 'MobileNet_V2_Weights',
 'MobileNet_V3_Large_Weights',
 'MobileNet_V3_Small_Weights',
 'RegNet',
 'RegNet_X_16GF_Weights'

In [959]:

class EfficientNet(nn.Module):
    def __init__(self, num_classes=7, pretrained=True, softmax = False):
        super(EfficientNet, self).__init__()
        self.softmax = softmax
        weights='ResNet50_Weights.DEFAULT'
        if not pretrained: weights = None
        self.resnet = torchvision.models.resnet50(weights=weights)
        self.resnet.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)
        
    def forward(self,x):
        x = self.resnet(x)
        return x

In [960]:
train_loader

In [961]:
model = models.efficientnet_b0()
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [962]:
model.classifier = nn.Sequential(
               nn.Linear(1280,7))

optimizer = optim.SGD(model.parameters(), lr=l_r)
for ep in range(e_pochs):
    for img, label in train_loader:
        #batch_size = img.shape[0]
        img.to(device)
        label.to(device)
        out = model(img)
        loss = criter(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss)
    print("Epoch: %d, Loss: %f" % (ep, float(loss)))


tensor(1.9169, grad_fn=<NllLossBackward0>)
tensor(1.9213, grad_fn=<NllLossBackward0>)
tensor(1.9132, grad_fn=<NllLossBackward0>)
tensor(1.9115, grad_fn=<NllLossBackward0>)
tensor(1.9015, grad_fn=<NllLossBackward0>)
tensor(1.9040, grad_fn=<NllLossBackward0>)
tensor(1.9009, grad_fn=<NllLossBackward0>)
tensor(1.9039, grad_fn=<NllLossBackward0>)
tensor(1.8871, grad_fn=<NllLossBackward0>)
tensor(1.8845, grad_fn=<NllLossBackward0>)
tensor(1.8819, grad_fn=<NllLossBackward0>)
tensor(1.8729, grad_fn=<NllLossBackward0>)
tensor(1.8727, grad_fn=<NllLossBackward0>)
tensor(1.8754, grad_fn=<NllLossBackward0>)
tensor(1.8755, grad_fn=<NllLossBackward0>)
tensor(1.7849, grad_fn=<NllLossBackward0>)
tensor(1.7946, grad_fn=<NllLossBackward0>)
tensor(1.8098, grad_fn=<NllLossBackward0>)
tensor(1.8012, grad_fn=<NllLossBackward0>)
tensor(1.7918, grad_fn=<NllLossBackward0>)
tensor(1.7935, grad_fn=<NllLossBackward0>)
tensor(1.7856, grad_fn=<NllLossBackward0>)
tensor(1.7811, grad_fn=<NllLossBackward0>)
tensor(1.77

Traceback (most recent call last):
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_5896\854161310.py", line 13, in <module>
    loss.backward()
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\autograd\__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Tra